Experiment on filtered labels. 
5-fold validation.

In [10]:
from gensim.models import Word2Vec
import annoy # fast approximate KNN
import numpy as np

In [5]:
!head ./UP000005640.goByGeneC

"V1" "V2"
"A0PJZ0" "MKLFGFRSRRGQTVLGSIDHLYTGSGYRIRYSELQKIHKAAVKGDAAEMERCLARRSGDLDALDKQHRTALHLACASGHVKVVTLLVNRKCQIDIYDKENRTPLIQAVHCQEEACAVILLEHGANPNLKDIYGNTALHYAVYSESTSLAEKLLFHGENIEALDKV" ""
"P30461" "MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRFITVGYVDDTQFVRFDSDATSPRMAPRAPWIEQEGPEYWDRETQISKTNTQTYRENLRTALRYYNQSEAGSHTWQTMYGCDLGPDGRLLRGHNQLAYDGKDYIALNEDLSSWTAADTAAQITQLKWEAARVAEQLRAYLEGECVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEPSSQSTVPIVGIVAGLAVLAVVVIGAVVAAVMCRRKSSGGKGGSYSQAACSDSAQGSDVSLTA" ""
"P18825" "MASPALAAALAVAAAAGPNASGAGERGSGGVANASGASWGPPRGQYSAGAVAGLAAVVGFLIVFTVVGNVLVVIAVLTSRALRAPQNLFLVSLASADILVATLVMPFSLANELMAYWYFGQVWCGVYLALDVLFCTSSIVHLCAISLDRYWSVTQAVEYNLKRTPRRVKATIVAVWLISAVISFPPLVSLYRQPDGAAYPQCGLNDETWYILSSCIGSFFAPCLIMGLVYARIYRVAKLRTRTLSEKRAPVGPDGASPTTENGLGAAAGAGENGHCAPPPADVEPDESSAAAERRRRRGALRRGGRRRAGAEGGAGGADGQGAGPGAAESGALTASRSPGPGGRLSRASSRSVEFFLSRRRRARSSVCRRKVAQAREKRFTFVLAVVMGVFVLCWFPFFFSYSLYGICREACQVPGPLFKFFF

In [11]:
with open('./good_labels') as inp:
    good_labels = set()
    for y in inp:
        good_labels.add(y.strip())

In [12]:
X = [] # sequence strings
Y = [] # list of labels for each sequence
all_y = set() # all the labels collected from the dataset
with open('./UP000005640.goByGeneC') as inp:
    inp.readline() # header
    for line in inp:
        tokens = map(lambda x: x.strip('"'), line.split(' ',2))
        X.append(tokens[1])
        object_labels = [t.strip().strip('"') for t in tokens[2].split(',') if len(t.strip())>0 and t.strip]
        Y.append([y for y in object_labels if y in good_labels])
        all_y = all_y.union(set(Y[-1]))

In [13]:
print 'example of X', X[:3]
print 'example of Y', Y[:3]
print 'example of set of labels', list(all_y)[:3]
print 'total sequences', len(X)
print 'total labels', len(all_y)

example of X ['MKLFGFRSRRGQTVLGSIDHLYTGSGYRIRYSELQKIHKAAVKGDAAEMERCLARRSGDLDALDKQHRTALHLACASGHVKVVTLLVNRKCQIDIYDKENRTPLIQAVHCQEEACAVILLEHGANPNLKDIYGNTALHYAVYSESTSLAEKLLFHGENIEALDKV', 'MRVTAPRTLLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRFITVGYVDDTQFVRFDSDATSPRMAPRAPWIEQEGPEYWDRETQISKTNTQTYRENLRTALRYYNQSEAGSHTWQTMYGCDLGPDGRLLRGHNQLAYDGKDYIALNEDLSSWTAADTAAQITQLKWEAARVAEQLRAYLEGECVEWLRRYLENGKETLQRADPPKTHVTHHPISDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEPSSQSTVPIVGIVAGLAVLAVVVIGAVVAAVMCRRKSSGGKGGSYSQAACSDSAQGSDVSLTA', 'MASPALAAALAVAAAAGPNASGAGERGSGGVANASGASWGPPRGQYSAGAVAGLAAVVGFLIVFTVVGNVLVVIAVLTSRALRAPQNLFLVSLASADILVATLVMPFSLANELMAYWYFGQVWCGVYLALDVLFCTSSIVHLCAISLDRYWSVTQAVEYNLKRTPRRVKATIVAVWLISAVISFPPLVSLYRQPDGAAYPQCGLNDETWYILSSCIGSFFAPCLIMGLVYARIYRVAKLRTRTLSEKRAPVGPDGASPTTENGLGAAAGAGENGHCAPPPADVEPDESSAAAERRRRRGALRRGGRRRAGAEGGAGGADGQGAGPGAAESGALTASRSPGPGGRLSRASSRSVEFFLSRRRRARSSVCRRKVAQAREKRFTFVLAVVMGVFVLCWFPFFFSYSLYGICREACQVPGPLFKFFFWIGYCNSSLNPVIYTVFNQDFRRSFKH

In [19]:
def cv_iter(w2v_params={}, trunc_num_train=-1, trunc_num_test=-1):    
    """
    one ieration of cross-validation
    w2v_params is a dictionary with parameters of gensim.Word2Vec
    """
    indices = range(len(X))
    np.random.shuffle(indices)

    Xtrain = np.array(X)[indices[:len(indices)/2]]
    Ytrain = np.array(Y)[indices[:len(indices)/2]]
    Xtest = np.array(X)[indices[len(indices)/2:]]
    Ytest = np.array(Y)[indices[len(indices)/2:]]
    # split all the strings into lists for word2vec traing
    Xtrain = [list(x[:trunc_num_train])for x in Xtrain]
    Xtest = [list(x) for x in Xtest]
    model = Word2Vec(Xtrain, **w2v_params)
    index = annoy.AnnoyIndex(w2v_params.get('size', 100)) # default value is 100 for gensim.Word2Vec
    for i in range(len(Xtrain)):
        v= np.mean([model[c] for c in Xtrain[i][:trunc_num_test]  if c in model], axis=0) # centroid
        index.add_item(i, v)
    index.build(100) # 100 is a number of trees in index, we can alibrate this value
    predicted = []
    for i in range(len(Xtest)):
        v= np.mean([model[c] for c in Xtest[i][:trunc_num_test] if c in model], axis=0)
        id = index.get_nns_by_vector(v,1)[0] #  finding 1 nearest neighbour
        predicted.append(id)
    tp = 0
    fp = 0
    true_label_num = 0
    for i in xrange(len(Xtest)):
        true = set(Ytest[i])
        true_label_num+=len(true)
        pred = set(Ytrain[predicted[i]])
        tp += len(true&pred)
        fp += len(pred) - len(true&pred)

    prec=tp*1.0/(tp+fp)
    rec = tp*1.0/true_label_num
    return prec, rec,  2 * prec * rec/ (prec+rec)
    

In [20]:
def cv(w2v_params = {}, cv_num = 5, trunc_num_train=-1, trunc_num_test=-1 ):
    """
    Cross-validation function.
    Returns list with precision, recall and F1 values
    """
    precs, recs, fs = [],[],[]
    for _ in range(cv_num):
        p,r,f = cv_iter(w2v_params, trunc_num_train, trunc_num_test)
        precs.append(p)
        recs.append(r)
        fs.append(f)
    return precs, recs, fs

In [21]:
p,r,f = cv(trunc_num_train=30, trunc_num_test=30)
np.mean(p), np.mean(r), np.mean(f)

(0.19801333252772998, 0.196039748835597, 0.19701227295607307)

In [22]:
p,r,f = cv(trunc_num_train=-1, trunc_num_test=30)
np.mean(p), np.mean(r), np.mean(f)

(0.20698461737507143, 0.20447192309871326, 0.20571312577256512)

In [23]:
p,r,f = cv(trunc_num_train=-1, trunc_num_test=10)
np.mean(p), np.mean(r), np.mean(f)

(0.18755352865326308, 0.18704630413653942, 0.18729574984411781)

In [24]:
p,r,f = cv(trunc_num_train=10, trunc_num_test=10)
np.mean(p), np.mean(r), np.mean(f)

(0.17654730227021329, 0.1737373235686667, 0.1751289463065961)

In [26]:
p,r,f = cv(trunc_num_train=200, trunc_num_test=200)
np.mean(p), np.mean(r), np.mean(f)

(0.2392785429491009, 0.2365239862657899, 0.23788249806664413)

In [27]:
p,r,f = cv(trunc_num_train=-1, trunc_num_test=200)
np.mean(p), np.mean(r), np.mean(f)

(0.2398334095723175, 0.24127898319906324, 0.2405217724238795)